In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [2]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0],dtype = np.complex128)
zeta = np.array([0,0],dtype = np.complex128)
theta = 0
psi = 0
theta1 = 0.3
psi1 = 0.5
cutoff_dim = 4
cutoff= 4
cutoff_plus = 10


In [3]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [4]:
old_state

array([[0.28975917+0.14717559j, 0.11199978+0.13633301j,
        0.27451839+0.00883316j, 0.22304876+0.08357372j],
       [0.17614783+0.25446674j, 0.12714215+0.19177718j,
        0.21560022+0.03086323j, 0.2982641 +0.05828827j],
       [0.23757876+0.03718481j, 0.13148204+0.12488271j,
        0.15735396+0.15456445j, 0.24292301+0.10612123j],
       [0.19982621+0.1440259j , 0.19369031+0.23258064j,
        0.23545511+0.14016976j, 0.07332695+0.01319642j]])

In [5]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff+1, dtype=dtype))

G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [6]:
mu

array([ 0.+0.j,  0.+0.j, -0.-0.j, -0.-0.j])

In [7]:
Sigma

array([[ 0.        +0.j        ,  0.        +0.j        ,
        -0.95533649+0.j        ,  0.25934338-0.14167993j],
       [ 0.        +0.j        ,  0.        +0.j        ,
        -0.25934338-0.14167993j, -0.95533649+0.j        ],
       [-0.95533649+0.j        , -0.25934338-0.14167993j,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.25934338-0.14167993j, -0.95533649+0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [8]:
G_00pq

array([[1.+0.j, 0.-0.j, 0.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j, 0.-0.j, 0.+0.j],
       [0.+0.j, 0.-0.j, 0.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j, 0.-0.j, 0.+0.j]])

In [51]:
# R_00^jk = a_dagger^j \G_00pq> b^k  * |old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)
sqrts = np.zeros((cutoff, cutoff), dtype=dtype)
for m in range(cutoff):
    for n in range(cutoff):
        sqrts[m,n]= np.sqrt(m+1)*np.sqrt(n+1)
        
for j in range(cutoff):
    for k in range(cutoff):
        if j==0 and k ==0:
            R[0,0,j,k] = np.sum(G_00pq*old_state)

        elif j>0 and k==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G_00pq[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        elif k>0 and j==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[:,k:] = G_00pq[:,:cutoff-k] * sqrt[1:cutoff-k+1]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        else:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,k:] = G_00pq[:cutoff-j,:cutoff-k] * sqrts[:cutoff-j,:cutoff-k]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)


print(R[0,0,:cutoff,:cutoff])    
#R_0n^jk
for n in range(1,cutoff):
    for j in range(0,cutoff):
        for k in range(0,cutoff):  
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]



for m in range(1,cutoff):
    for n in range(0,cutoff):
        for j in range(0,cutoff):
            for k in range(0,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]


[[0.28975917+0.14717559j 0.11199978+0.13633301j 0.27451839+0.00883316j
  0.22304876+0.08357372j]
 [0.17614783+0.25446674j 0.12714215+0.19177718j 0.21560022+0.03086323j
  0.2982641 +0.05828827j]
 [0.23757876+0.03718481j 0.13148204+0.12488271j 0.15735396+0.15456445j
  0.24292301+0.10612123j]
 [0.19982621+0.1440259j  0.19369031+0.23258064j 0.23545511+0.14016976j
  0.07332695+0.01319642j]]


In [90]:
#Use g_matrix to caompare the R matrix
j=0
k=1
m=0
n=1
G22 = G[m,n,:,:]
if j==0 and k ==0:
    a = np.sum(G22*old_state)

elif j>0 and k==0:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[j:,:] = G22[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
    a = np.sum(G_00pq2*old_state)

elif k>0 and j==0:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[:,k:] = G22[:,:cutoff-k] * sqrt[1:cutoff-k+1]
    a = np.sum(G_00pq2*old_state)

else:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[j:,k:] = G22[:cutoff-j,:cutoff-k]  * sqrts[:cutoff-j,:cutoff-k]
    a = np.sum(G_00pq2*old_state)

In [91]:
a

(0.3766905177579023+0.07968366781126048j)

In [52]:
R[:,:,:cutoff,:cutoff]

array([[[[ 2.89759170e-01+0.14717559j,  1.11999775e-01+0.13633301j,
           2.74518389e-01+0.00883316j,  2.23048757e-01+0.08357372j],
         [ 1.76147833e-01+0.25446674j,  1.27142154e-01+0.19177718j,
           2.15600219e-01+0.03086323j,  2.98264102e-01+0.05828827j],
         [ 2.37578756e-01+0.03718481j,  1.31482040e-01+0.12488271j,
           1.57353965e-01+0.15456445j,  2.42923009e-01+0.10612123j],
         [ 1.99826205e-01+0.1440259j ,  1.93690305e-01+0.23258064j,
           2.35455115e-01+0.14016976j,  7.33269523e-02+0.01319642j]],

        [[ 1.16627416e-01+0.22119478j,  2.68059932e-01+0.07618827j,
           2.64628406e-01+0.11839143j,  6.90945423e-02+0.05737471j],
         [ 1.77809675e-01+0.22651551j,  2.22376378e-01+0.08050064j,
           3.03852608e-01+0.11806408j,  4.79652259e-02+0.06193915j],
         [ 1.57027614e-01+0.18496854j,  1.67606272e-01+0.23542134j,
           2.73277698e-01+0.17109284j,  1.71471914e-02+0.01381136j],
         [ 1.85039416e-01+0.22219278j, 

In [53]:
R[:,:,0,0]

array([[ 0.28975917+0.14717559j,  0.11662742+0.22119478j,
         0.19099555+0.11081975j,  0.1321641 +0.09324692j],
       [ 0.1199184 +0.22361242j,  0.08955415+0.23461833j,
         0.08189398+0.09900003j,  0.08196575+0.10164818j],
       [ 0.08168256-0.03283175j,  0.02299311+0.12392952j,
        -0.01282586+0.13092028j,  0.04942346+0.05186833j],
       [ 0.02674843+0.02133457j,  0.0062675 +0.01846275j,
        -0.02117114+0.01863446j,  0.00204704+0.0003684j ]])

In [67]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [55]:
np.einsum("cd,abcd->ab",old_state,G)

array([[ 0.28975917+0.14717559j,  0.11662742+0.22119478j,
         0.26686128+0.11880853j,  0.26769113+0.16495421j],
       [ 0.1199184 +0.22361242j,  0.08318322+0.26623847j,
         0.06844853+0.16166241j,  0.19256612+0.2219097j ],
       [ 0.14917482-0.02867838j,  0.06516384+0.21042844j,
        -0.04986888+0.30256293j,  0.17120788+0.17967718j],
       [ 0.11038458+0.0856091j ,  0.04257111+0.05294688j,
        -0.073339  +0.06455168j,  0.01228223+0.00221039j]])

In [44]:
np.sum(G[0,0,:,:]*old_state)

(0.2897591695977617+0.14717559391129217j)

In [23]:
np.sum(G[1,1,:,:]*old_state)

(0.08318321963269501+0.26623847379201515j)

In [16]:
G[0,2,:,:]*old_state

array([[0.        +0.j        , 0.        +0.j        ,
        0.2505441 +0.00806174j, 0.        +0.j        ],
       [0.        +0.j        , 0.00783946+0.09153313j,
        0.        +0.j        , 0.        +0.j        ],
       [0.00847772+0.01921366j, 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ]])